In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2884663/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_14_5_1,0.971860,0.930769,0.921247,0.927427,0.031142,0.093945,0.070437,0.082882,"Hidden Size=[40], regularizer=0.02, learning_r..."
1,model_14_5_2,0.969900,0.927372,0.902468,0.918370,0.033312,0.098554,0.087232,0.093226,"Hidden Size=[40], regularizer=0.02, learning_r..."
2,model_14_5_0,0.969711,0.937037,0.919341,0.930668,0.033521,0.085438,0.072141,0.079181,"Hidden Size=[40], regularizer=0.02, learning_r..."
3,model_14_5_3,0.968249,0.924075,0.889890,0.911660,0.035139,0.103028,0.098482,0.100889,"Hidden Size=[40], regularizer=0.02, learning_r..."
4,model_14_5_4,0.968009,0.924416,0.887165,0.910870,0.035404,0.102565,0.100919,0.101791,"Hidden Size=[40], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
1751,model_8_1_3,0.831869,0.912405,0.772307,0.822014,0.186071,0.116535,0.670178,0.377073,"Hidden Size=[30], regularizer=0.1, learning_ra..."
1752,model_20_1_0,0.831781,0.493034,0.768515,0.664578,0.186169,0.680358,0.126368,0.419657,"Hidden Size=[55], regularizer=0.1, learning_ra..."
1753,model_18_7_0,0.831762,0.638049,0.926949,0.829065,0.186190,0.417282,0.184808,0.307883,"Hidden Size=[55], regularizer=0.02, learning_r..."
1758,model_25_7_11,0.831416,0.874364,0.814441,0.850398,0.186573,0.234011,0.513279,0.365431,"Hidden Size=[70], regularizer=0.02, learning_r..."
